In [56]:
import requests
import bs4
import pandas as pd

In [57]:
url_base = 'https://www.cars.com/for-sale/searchresults.action/?page=1&perPage=100&rd=99999&searchSource=GN_REFINEMENT&showMore=true&sort=relevance&stkTypId=28881&zc=92093&userSetIxt=true'
response = requests.get(url_base)
html = bs4.BeautifulSoup(response.text, "lxml")

In [5]:
isinstance(6.3, (int, float))

True

In [58]:
def grab_data_from_given_url(url, lst):
    response = requests.get(url)
    html = bs4.BeautifulSoup(response.text, "lxml")
    tmp_lst = html.find_all("div", id = "srp-listing-rows-container")
    if(len(tmp_lst) == 0):
        print('Cannot search page: ' + url)
        return
    container = html.find_all("div", id = "srp-listing-rows-container")[0]
    for child in container.children:
        car = {}
        if type(child) is not bs4.element.NavigableString and child.name == 'div':
            if child.has_attr('class') and child.a:
                if child.a and child.a.div:
                    for ch in child.a.div.children:
                        if type(ch) is bs4.element.Tag and ch.has_attr('class') and ch['class'][0] == 'listing-row__details':
                            car['Name'] = ch.h2.text.strip()
                            for item in ch.find_all('span'):
                                if type(item) is bs4.element.Tag and item.has_attr('class') and item['class'][0] == 'listing-row__mileage':
                                    car['Mileage'] = item.text.strip()
                                elif type(item) is bs4.element.Tag and item.has_attr('class') and item['class'][0] == 'listing-row__price':
                                    car['Price'] = item.text.strip()
                else:
                    print('Cannot search page: ' + url)
                    return 
                        
        if len(car) > 0:
            lst.append(car)


In [59]:
url_base = 'https://www.cars.com/for-sale/searchresults.action/?page=1&perPage=20&rd=99999&searchSource=GN_REFINEMENT&showMore=true&sort=relevance&stkTypId=28881&zc=92093&userSetIxt=true'
url_head = 'https://www.cars.com/for-sale/searchresults.action/?'
url_tail = '&perPage=100&rd=99999&searchSource=GN_REFINEMENT&showMore=true&sort=relevance&stkTypId=28881&zc=92093&userSetIxt=true'

brand_dict = {}

def generate_brand_keys():
    response = requests.get(url_base)
    html = bs4.BeautifulSoup(response.text, "lxml")
    make_lst = html.find_all('input', attrs={"name": "mkId"})
    for brand in make_lst:
        
        brand_name = brand.parent.label.text.strip()
        mkid = brand['value']
        brand_dict[brand_name] = mkid
        
generate_brand_keys()      
    

In [60]:
def generate_model_keys(url_tmp, dic):
    response = requests.get(url_tmp)
    html = bs4.BeautifulSoup(response.text, "lxml")
    md_lst = html.find_all('input', attrs={"name": "mdId"})
    for md in md_lst:
        
        md_name = md.parent.label.text.strip()
        mdid = md['value']
        dic[md_name] = mdid

# brand_already_searched = ['AC', 'Acura', 'Alfa Romeo', 'Am General', \
#                           'American Motors', 'Aston Martin', 'Audi', 'Austin-Healey', \
#                           'Avanti Motors', 'BMW', 'Bentley', 'Bricklin', 'Bugatti', 'Buick', \
#                           'Cadillac', 'Chevrolet', 'Chrysler', 'Citroen', 'Daewoo', 'Datsun', \
#                           'DeTomaso', 'Delorean', 'Dodge', 'Eagle', 'Edsel', 'FIAT', 'Ferrari', \
#                           'Fisker', 'Ford', 'GMC', 'Genesis', 'Geo', 'Honda', 'Hudson', 'Hummer', \
#                           'Hyundai', 'INFINITI', 'International', 'Isuzu', 'Jaguar', \
#                           'Mercedes-Benz', 'McLaren', 'Mazda', 'Maybach', 'Maserati', 'MINI', \
#                           'MG', 'Lotus', 'Lincoln', 'Lexus', 'Land Rover', 'Lamborghini', 'Koenigsegg', \
#                           'Jensen', 'Karma', 'Kia', 'Jeep', 'Mercury', 'Mitsubishi', 'Morris', 'Nash']


In [61]:
def launch_search_given_id(url, lst, make_id, md_id):
    response = requests.get(url)
    html = bs4.BeautifulSoup(response.text, "lxml")
    lst_tmp = html.find_all('span', attrs={'class':"count"})
    if len(lst_tmp) == 0:
        print('Cannot search page: ' + url)
        return
    found_count = int(lst_tmp[0].text.replace(',', ''))
    num_of_pages = found_count//100 + 1
    for page_num in range(1, num_of_pages + 1):
        if (md_id == ''):
            grab_data_from_given_url(url_head + 'mkId=' + make_id + "&" + "page=" \
                                         + str(page_num) + url_tail, lst)
        else:
            grab_data_from_given_url(url_head + "mdId=" + md_id + "&" + 'mkId=' + make_id + "&" + "page=" \
                                         + str(page_num) + url_tail, lst)

def launch_search():
    for brand in brand_dict:
        if brand in brand_already_searched:
            continue
        brand_already_searched.append(brand)
        print("Now searching: " + brand)
        make_id = brand_dict[brand]
        url_tmp = url_head + 'mkId=' + make_id + "&" + "page=1" + url_tail
        response = requests.get(url_tmp)
        html = bs4.BeautifulSoup(response.text, "lxml")

        # See how many cars we found.
        found_count = int(html.find_all('span', attrs={'class':"count"})[0].text.replace(',', ''))
        
        # If less then 5000, start the search right now
        if found_count <= 5000:
            list_of_cars = []
            md_id = ''
            launch_search_given_id(url_tmp, list_of_cars, make_id, md_id)
            df_car = pd.DataFrame(list_of_cars, columns=['Name', 'Mileage', 'Price'])
            file_name = brand + '.csv'
            file_name = file_name.replace('/', ' ')
            df_car.to_csv(file_name)
        else:
            md_dict = {}
            generate_model_keys(url_tmp, md_dict)
            
            for md in md_dict:
                print("Now searching model: " + md)
                md_id = md_dict[md]
                url_tmp = url_head + "mdId=" + md_id + "&" + 'mkId=' + make_id + "&" + "page=1" + url_tail
                response = requests.get(url_tmp)
                html = bs4.BeautifulSoup(response.text, "lxml")
                list_of_cars = []
                launch_search_given_id(url_tmp, list_of_cars, make_id, md_id)
                df_car = pd.DataFrame(list_of_cars, columns=['Name', 'Mileage', 'Price'])
                file_name = brand + "-" + md + '.csv'
                file_name = file_name.replace('/', ' ')
                df_car.to_csv(file_name)




In [54]:
launch_search()

Now searching: Subaru
Now searching model: Ascent
Now searching model: B9 Tribeca
Now searching model: BRZ
Now searching model: Baja
Now searching model: Brat
Now searching model: Crosstrek
Now searching model: Crosstrek Hybrid
Now searching model: Forester
Now searching model: GL
Now searching model: Impreza
Now searching model: Impreza Outback Sport
Now searching model: Impreza WRX
Now searching model: Impreza WRX STi
Now searching model: Legacy
Now searching model: Outback
Now searching model: SVX
Now searching model: Tribeca
Now searching model: WRX
Now searching model: WRX STI
Now searching model: XT
Now searching model: XV Crosstrek
Now searching model: XV Crosstrek Hybrid
Now searching model: Unknown
Now searching: Sunbeam
Now searching: Suzuki
Now searching: Tesla
Now searching: Toyota
Now searching model: 4Runner
Now searching model: 86
Now searching model: Avalon
Now searching model: Avalon Hybrid
Now searching model: C-HR
Now searching model: Camry
Now searching model: Camry

In [55]:
brand_dict

{'AC': '20046',
 'Acura': '20001',
 'Alfa Romeo': '20047',
 'Am General': '20002',
 'American Motors': '20048',
 'Aston Martin': '20003',
 'Audi': '20049',
 'Austin-Healey': '20004',
 'Avanti Motors': '20050',
 'BMW': '20005',
 'Bentley': '20051',
 'Bricklin': '36354988',
 'Bugatti': '33583',
 'Buick': '20006',
 'Cadillac': '20052',
 'Chevrolet': '20053',
 'Chrysler': '20008',
 'Citroen': '20054',
 'Daewoo': '20009',
 'Datsun': '20010',
 'DeTomaso': '20056',
 'Delorean': '20011',
 'Dodge': '20012',
 'Eagle': '20058',
 'Edsel': '20013',
 'FIAT': '20060',
 'Ferrari': '20014',
 'Fisker': '41703',
 'Ford': '20015',
 'GMC': '20061',
 'Genesis': '35354491',
 'Geo': '20062',
 'Honda': '20017',
 'Hudson': '20063',
 'Hummer': '20018',
 'Hyundai': '20064',
 'INFINITI': '20019',
 'International': '20065',
 'Isuzu': '20020',
 'Jaguar': '20066',
 'Jeep': '20021',
 'Jensen': '20067',
 'Karma': '36365359',
 'Kia': '20068',
 'Koenigsegg': '33663',
 'Lamborghini': '20069',
 'Land Rover': '20024',
 'Lex

In [ ]:
def launch_search_given_id(url, lst, make_id, md_id):
    response = requests.get(url)
    html = bs4.BeautifulSoup(response.text, "lxml")
    lst_tmp = html.find_all('span', attrs={'class':"count"})
    if len(lst_tmp) == 0:
        print('Cannot search page: ' + url)
        return
    found_count = int(lst_tmp[0].text.replace(',', ''))
    num_of_pages = found_count//100 + 1
    for page_num in range(1, num_of_pages + 1):
        if (md_id == ''):
            grab_data_from_given_url(url_head + 'mkId=' + make_id + "&" + "page=" \
                                         + str(page_num) + url_tail, lst)
        else:
            grab_data_from_given_url(url_head + "mdId=" + md_id + "&" + 'mkId=' + make_id + "&" + "page=" \
                                         + str(page_num) + url_tail, lst)

def launch_search():
    for brand in brand_dict:
        if brand == 'Nissan':
            break
        print("Now searching: " + brand)
        make_id = brand_dict[brand]
        url_tmp = url_head + 'mkId=' + make_id + "&" + "page=1" + url_tail
        response = requests.get(url_tmp)
        html = bs4.BeautifulSoup(response.text, "lxml")

        # See how many cars we found.
        found_count = int(html.find_all('span', attrs={'class':"count"})[0].text.replace(',', ''))
        
        # If less then 5000, start the search right now
        if found_count <= 5000:
            list_of_cars = []
            md_id = ''
            launch_search_given_id(url_tmp, list_of_cars, make_id, md_id)
            df_car = pd.DataFrame(list_of_cars, columns=['Name', 'Mileage', 'Price'])
            file_name = '(New)' + brand + '.csv'
            file_name = file_name.replace('/', ' ')
            df_car.to_csv(file_name)
        else:
            md_dict = {}
            generate_model_keys(url_tmp, md_dict)
            
            for md in md_dict:
                print("Now searching model: " + md)
                md_id = md_dict[md]
                url_tmp = url_head + "mdId=" + md_id + "&" + 'mkId=' + make_id + "&" + "page=1" + url_tail
                response = requests.get(url_tmp)
                html = bs4.BeautifulSoup(response.text, "lxml")
                list_of_cars = []
                launch_search_given_id(url_tmp, list_of_cars, make_id, md_id)
                df_car = pd.DataFrame(list_of_cars, columns=['Name', 'Mileage', 'Price'])
                file_name = '(New)' + brand + "-" + md + '.csv'
                file_name = file_name.replace('/', ' ')
                df_car.to_csv(file_name)


In [ ]:
launch_search()

In [27]:
s = 'as'
s.replace('a', ' ')

' s'

In [26]:
s

' s'

In [46]:
html.find_all('input', attrs={"name": "mkId"})[0]['value']

'20046'

In [5]:
html.find_all('span', attrs={'class':"count"})[0]

<span class="count">1,908,046</span>

In [89]:
html.find_all("href", class_ = "js-last-page")

[]

In [ ]:
 html.find_all('input', attrs={"name": "mkId"})[0].parent.label.text.strip()

In [71]:
html.find_all('span', attrs={'class':"count"})[0].text.replace(',', '')

'1905220'

In [ ]:
https://www.cars.com/for-sale/searchresults.action/?mdId=21138&mkId=20015&
    mlgId=28863&page=1&perPage=20&rd=99999&searchSource=GN_REFINEMENT&
    shippable-dealers-checkbox=true&showMore=true&sort=relevance&stkTypId=28881&zc=92093&localVehicles=false

In [62]:
a = {'Lincoln': '20025',
 'Lotus': '20071',
 'MG': '20026',
 'MINI': '20075',
 'Maserati': '20072',
 'Maybach': '20027',
 'Mazda': '20073',
 'McLaren': '47903',
 'Mercedes-Benz': '20028',
 'Mercury': '20074',
 'Mitsubishi': '20030',
 'Morris': '20225',
 'Nash': '20031'}

In [63]:
def launch_search_copy():
    for brand in a:
        print("Now searching: " + brand)
        make_id = brand_dict[brand]
        url_tmp = url_head + 'mkId=' + make_id + "&" + "page=1" + url_tail
        response = requests.get(url_tmp)
        html = bs4.BeautifulSoup(response.text, "lxml")

        # See how many cars we found.
        found_count = int(html.find_all('span', attrs={'class':"count"})[0].text.replace(',', ''))
        
        # If less then 5000, start the search right now
        if found_count <= 5000:
            list_of_cars = []
            md_id = ''
            launch_search_given_id(url_tmp, list_of_cars, make_id, md_id)
            df_car = pd.DataFrame(list_of_cars, columns=['Name', 'Mileage', 'Price'])
            file_name = '(New)' + brand + '.csv'
            file_name = file_name.replace('/', ' ')
            df_car.to_csv(file_name)
        else:
            md_dict = {}
            generate_model_keys(url_tmp, md_dict)
            
            for md in md_dict:
                print("Now searching model: " + md)
                md_id = md_dict[md]
                url_tmp = url_head + "mdId=" + md_id + "&" + 'mkId=' + make_id + "&" + "page=1" + url_tail
                response = requests.get(url_tmp)
                html = bs4.BeautifulSoup(response.text, "lxml")
                list_of_cars = []
                launch_search_given_id(url_tmp, list_of_cars, make_id, md_id)
                df_car = pd.DataFrame(list_of_cars, columns=['Name', 'Mileage', 'Price'])
                file_name = '(New)' + brand + "-" + md + '.csv'
                file_name = file_name.replace('/', ' ')
                df_car.to_csv(file_name)



In [64]:
launch_search_copy()

Now searching: Lincoln
Now searching model: Aviator
Now searching model: Blackwood
Now searching model: Capri
Now searching model: Continental
Now searching model: LS
Now searching model: MKC
Now searching model: MKS
Now searching model: MKT
Now searching model: MKX
Now searching model: MKZ
Now searching model: MKZ Hybrid
Now searching model: Mark III
Now searching model: Mark IV
Now searching model: Mark LT
Now searching model: Mark V
Now searching model: Mark VI
Now searching model: Mark VII
Now searching model: Mark VIII
Now searching model: Nautilus
Now searching model: Navigator
Now searching model: Navigator L
Now searching model: Town Car
Now searching model: Zephyr
Now searching model: Unknown
Now searching: Lotus
Now searching: MG
Now searching: MINI
Now searching model: Clubman
Now searching model: Convertible
Now searching model: Cooper
Now searching model: Cooper Clubman
Now searching model: Cooper Countryman
Now searching model: Cooper S
Now searching model: Cooper S Clubm

In [67]:
import os
for filename in os.listdir("."):
    if filename.startswith("(New)"):
        print(filename)
        os.rename(filename, filename[5:])


(New)INFINITI-G25x.csv
(New)Chevrolet-Avalanche.csv
(New)MINI-Unknown.csv
(New)MINI-Cooper S Clubman.csv
(New)Lexus-SC 300.csv
(New)Mercedes-Benz-GLE 550e.csv
(New)Lexus-RC 300.csv
(New)GMC-Savana 3500.csv
(New)BMW-325.csv
(New)Dodge-Panel.csv
(New)Dodge-W250.csv
(New)BMW-Z4 M.csv
(New)Lincoln-Blackwood.csv
(New)Audi-TT RS.csv
(New)Mazda-323.csv
(New)Lexus-NX 300.csv
(New)Kia-Spectra5.csv
(New)Lotus.csv
(New)Audi-RS6.csv
(New)Ford-Expedition Max.csv
(New)Mercedes-Benz-AMG CLS.csv
(New)Chrysler-Fifth Avenue.csv
(New)Chrysler-Unknown.csv
(New)BMW-735.csv
(New)Honda-S2000.csv
(New)Ford-Pickup.csv
(New)Chevrolet-Cruze.csv
(New)Dodge-Caliber.csv
(New)Audi-A8.csv
(New)Lincoln-Aviator.csv
(New)Ford-EcoSport.csv
(New)Cadillac-Unknown.csv
(New)Kia-Niro.csv
(New)Buick-Super.csv
(New)Mercedes-Benz-GLS 450.csv
(New)Audi-Q5 hybrid.csv
(New)Dodge-Pickup.csv
(New)GMC-Sprint.csv
(New)Mazda-929.csv
(New)Isuzu.csv
(New)INFINITI-M35x.csv
(New)BMW-Unknown.csv
(New)Alfa Romeo.csv
(New)Mercedes-Benz-Maybach

(New)Lexus-RX 330.csv
(New)Mercedes-Benz-C-Class.csv
(New)Cadillac-Series 62.csv
(New)BMW-M5.csv
(New)INFINITI-Q45.csv
(New)BMW-430.csv
(New)Mercedes-Benz-250SL.csv
(New)Mercedes-Benz-AMG E 43.csv
(New)Honda-Fit.csv
(New)Chevrolet-Malibu Maxx.csv
(New)Mercedes-Benz-220SE.csv
(New)Mercedes-Benz-AMG GLS 63.csv
(New)Dodge-Daytona.csv
(New)Ford-Crown Victoria.csv
(New)BMW-X4.csv
(New)Chevrolet-Sonic.csv
(New)INFINITI-M37.csv
(New)Ford-Focus ST.csv
(New)MINI-Cooper S.csv
(New)Chevrolet-Express 3500.csv
(New)Jaguar-XJ8.csv
(New)BMW-428 Gran Coupe.csv
(New)Cadillac-Escalade ESV.csv
(New)Ford-Thunderbird.csv
(New)Chevrolet-Chevy II.csv
(New)Mitsubishi-Lancer Evolution.csv
(New)Lexus-SC 430.csv
(New)Chrysler-Voyager.csv
(New)Audi-5000.csv
(New)Lincoln-Mark VII.csv
(New)INFINITI-Q60.csv
(New)Lexus-GX 460.csv
(New)Ford-Fusion Hybrid.csv
(New)Chevrolet-Independence.csv
(New)Ford-Ranch.csv
(New)BMW-640 Gran Turismo.csv
(New)Honda-Element.csv
(New)Mercedes-Benz-AMG G 63.csv
(New)INFINITI-G25.csv
(Ne

In [6]:
import os
import pandas as pd
car_count = 0
for filename in os.listdir("./Data"):
    df = pd.read_csv("./Data/" + filename)
    car_count += len(df)
#     print(car_count)
print(car_count)

27
1754
1761
1838
1839
1842
2060
2352
29852
51252
53088
53198
53265
53443
56393
58334
58342
58474
58495
58498
81298
81678
83638
83795
96495
96553
98161
98214
98280
118180
118181
118185
119181
121659
121668
121672
143472
143596
143736
143744
143970
159170
159215
160638
168438
168995
169014
169015
169020
171684
171939
171946
172294
172297
175081
175241
175243
175520
178000
188200
188247
188809
188826
188838
188880
188881
188937
189055
189213
189253
191936
191940
202240
202266
202271
202277
205261
205403
205611
205638
205761
205762
205773
205954
205966
206002
206003
206460
206632
207214
211039
211099
211101
212063
212182
212530
213345
213371
213537
213538
213602
213703
213704
213709
213778
213786
213795
214289
223989
225550
225553
225742
225743
225746
250246
250384
250408
250460
251207
255538
255571
255815
255880
260573
260580
264350
264368
264372
272972
272976
273092
273093
273109
273125
280725
280749
280785
280787
280788
280851
288451
288455
289045
289193
289898
306698
306768
306795
306

1835363
1835567
1835755
1838321
1843721
1843757
1843785
1843786
1843839
1849239
1849973
1850030
1850301
1851067
1851166
1851178
1851228
1851542
1851551
1851765
1853492
1853493
1854501
1855047
1855669
1855673
1856136
1856946
1856949
1856984
1856990
1858348
1858363
1859681
1859714
1859874
1859952
1859959
1859961
1860056
1860057
1860060
1860178
1860320
1860383
1860393
1860425
1860772
1861460
1862413
1862649
1862664
1862832
1862854
1863760
1863763
1864062
1865992
1867584
1867611
1867739
1867816
1868580
1875180
1876151
1877648
1903048
1903050
1903051
1903236
1903440
1903454
1903519
1903519


In [7]:
print(car_count)

1903519
